# Import

In [15]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score


from sklearn.linear_model import HuberRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, PowerTransformer

import pickle

from imblearn.over_sampling import SMOTE

import matplotlib.pylab as plt

# model 
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, \
    TweedieRegressor,  SGDRegressor, RANSACRegressor, TheilSenRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor, \
    RadiusNeighborsRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

# Load data

In [16]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
data = pd.concat([train, test])

# Data Preprocessing

### Remove the data that is not going to use

In [17]:
# Drop attrubute column, 
# Since we can use product_code to identify different product
data = data.drop([col for col in data.columns 
                  if col.startswith('attribute')], axis=1)
data.columns

Index(['id', 'product_code', 'loading', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure'],
      dtype='object')

### Fill NaN according to most correlated field, if non matching, using KNN instad.

> Find correlated coefficiency

In [18]:
candidate_columns = [col for col in data.columns 
                     if col.startswith('measurement_') and 
                     int(col.split('measurement_')[1]) > 2] 
most_correlated_columns \
    = {productId:[] for productId in data.product_code.unique()}

#for each product
for product_code in data.product_code.unique():
    productData = data[data.product_code==product_code]

    product_correlation = {}
    for col in candidate_columns:
        productDataColumn = productData[candidate_columns]

        #calculate correlation for specific column for specific product
        correlations = productDataColumn.corr()[col] \
                                        .sort_values(ascending=False) 

        #take the most 5 correlation(except itself)
        most_correlations = correlations[1:2] 
        product_correlation[col] = most_correlations.index.tolist()
    most_correlated_columns[product_code] = product_correlation

In [19]:
len(most_correlated_columns)

9

> Fill Null

In [20]:
# use these columns as the reference columns of KNN
reference_columns = [col for col in data.columns 
                    if col.startswith('measurement') or col=='loading']
null_columns = [col for col in reference_columns \
                    if data[col].isnull().any()]
print(reference_columns)
print(null_columns)

['loading', 'measurement_0', 'measurement_1', 'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17']
['loading', 'measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17']


In [21]:
for product_code in data.product_code.unique():
    for current_column, correlated_columns in \
        most_correlated_columns[product_code].items():
        data_part = data[data.product_code==product_code] \
                        [correlated_columns+[current_column, 'id']]

        complete_data_part = data_part.dropna(how='any')
        predictor = HuberRegressor(epsilon=1.95)
        predictor.fit(complete_data_part[correlated_columns], 
                      complete_data_part[current_column])
    
        na_data_dart = data_part[data_part[current_column].isnull() & 
                                 (~data_part[correlated_columns]
                                    .isnull().any(axis=1))]
        data.loc[data.id.isin(na_data_dart.id), current_column] = \
            predictor.predict(na_data_dart[correlated_columns])

    # others NA columns:
    na_data_index = (data.product_code==product_code, reference_columns)
    knn = KNNImputer(n_neighbors=3)
    data.loc[na_data_index] = knn.fit_transform(data.loc[na_data_index])

In [22]:
data.isnull().sum()

id                    0
product_code          0
loading               0
measurement_0         0
measurement_1         0
measurement_2         0
measurement_3         0
measurement_4         0
measurement_5         0
measurement_6         0
measurement_7         0
measurement_8         0
measurement_9         0
measurement_10        0
measurement_11        0
measurement_12        0
measurement_13        0
measurement_14        0
measurement_15        0
measurement_16        0
measurement_17        0
failure           20775
dtype: int64

# Prepare trainX and trainY

In [23]:
df_train = data.iloc[:train.shape[0],:]
df_test = data.iloc[train.shape[0]:,:]

In [24]:
features = ['loading',
            'measurement_17',
            'measurement_1',
            'measurement_2',
            'measurement_11',
            'measurement_13',
            'measurement_4',
            'measurement_7',
            'measurement_3',
            'measurement_5',
            'measurement_0',
            'measurement_9',
            'measurement_12',
            'measurement_6',
            'measurement_10',      
            'measurement_16',
            'measurement_8',
            'measurement_14',
            'measurement_15',        
]
features = features[:4]

In [25]:
train_x = df_train[features]
train_y = df_train['failure']
test_x = df_test[features]

## Testing

In [26]:
with open('/content/model.pkl','rb') as modelFile:
    clf = pickle.loads(modelFile.read())

In [27]:
print(f"start testing...")

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

try:
    test_y = clf.predict_proba(test_x)[:,1]
except:
    test_y = clf.predict(test_x).reshape(-1)
print(f"finish!")

start testing...
finish!


> Save model

In [28]:
output = pd.read_csv('/content/sample_submission.csv')
output['failure'] = test_y

output.to_csv("/content/submission.csv", index=False)